# LSTM의 `return_sequences`, `return_state` 이해하기

In [ ]:
import numpy as np
from tensorflow.keras.layers import LSTM

In [ ]:
sample_data = np.random.randn(1, 4, 5) # (Batch, Timesteps, Embedding dim)

case 1. `return_sequences=False`, `return_state=False`
* 제일 마지막 `hidden_state`만 반환된다.
* 이 옵션이 기본값

In [ ]:
last_hidden_state = LSTM(3, return_sequences=False, return_state=False)(sample_data)
print(last_hidden_state)

tf.Tensor([[-0.24535894 -0.27312386 -0.0538178 ]], shape=(1, 3), dtype=float32)


case 2. `return_sequences=False`, `return_state=True`
* `hidden_states` : 제일 마지막 state. `last_hidden_state`와 같다.
* `last_hidden_state` : 제일 마지막 `hidden_state`
* `last_cell_state` : 제일 마지막 `cell_state`

In [ ]:
hidden_states, last_hidden_state, last_cell_state = LSTM(3, return_sequences=False, return_state=True)(sample_data)

print("hidden_states : {}".format(hidden_states.__repr__()))
print("last_hidden_state : {}".format(last_hidden_state.__repr__()))
print("last_cell_state : {}".format(last_cell_state.__repr__()))

hidden_states : <tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[-0.10527468, -0.12779412, -0.19543421]], dtype=float32)>
last_hidden_state : <tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[-0.10527468, -0.12779412, -0.19543421]], dtype=float32)>
last_cell_state : <tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[-0.17647648, -0.4138779 , -0.28452817]], dtype=float32)>


case 3. `return_sequences=True`, `return_state=False`
* `hidden_states` 만 등장한다.
* 매 시퀀스의 출력값만 등장

In [ ]:
hidden_states = LSTM(3, return_sequences=True, return_state=False)(sample_data)
print("hidden_states : {}".format(hidden_states.__repr__()))

hidden_states : <tf.Tensor: shape=(1, 4, 3), dtype=float32, numpy=
array([[[-0.10560024, -0.14313582, -0.27681196],
        [-0.09291577, -0.25481808, -0.01195437],
        [-0.31565538, -0.2275723 , -0.07195552],
        [-0.51375395, -0.1825685 , -0.39207292]]], dtype=float32)>


case 4. `return_sequences=True`, `return_state=True`
* `hidden_states`, `last_hidden_state`, `last_cell_state`가 모두 등장

In [ ]:
hidden_states, last_hidden_state, last_cell_state = LSTM(3, return_sequences=True, return_state=True)(sample_data)

print("hidden_states : {}".format(hidden_states.__repr__()))
print("last_hidden_state : {}".format(last_hidden_state.__repr__()))
print("last_cell_state : {}".format(last_cell_state.__repr__()))

hidden_states : <tf.Tensor: shape=(1, 4, 3), dtype=float32, numpy=
array([[[ 0.00815869,  0.10324696, -0.12757036],
        [-0.03125606,  0.00970532, -0.17458802],
        [-0.05928417,  0.19194022, -0.3302095 ],
        [ 0.08037872,  0.49555954, -0.15872242]]], dtype=float32)>
last_hidden_state : <tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[ 0.08037872,  0.49555954, -0.15872242]], dtype=float32)>
last_cell_state : <tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[ 0.13251474,  0.63943154, -1.1759439 ]], dtype=float32)>


# Seq2Seq 구현하기

## Encoder

In [ ]:
import tensorflow as tf

from tensorflow.keras.layers import Embedding, LSTM

class Encoder(tf.keras.Model):

  def __init__(self, vocab_size=2000):
    super(Encoder, self).__init__()
    # 레이어 구성 - Embedding Layer - LSTM
    self.emb = Embedding(vocab_size, 128)
    self.lstm = LSTM(512, return_sequences=False, return_state=True)

  def call(self, x):
    x = self.emb(x)

    # Encoder에서는 Context Vector만 만들면 되기 때문에 hidden_states에 대한 리턴은 필요 없다.
    _, h, c = self.lstm(x)

    return h, c  # Context Vector

## Decoder

In [ ]:
from tensorflow.keras.layers import Dense

class Decoder(tf.keras.Model):

  def __init__(self, vocab_size=2000):
    super(Decoder, self).__init__()

    self.emb = Embedding(vocab_size, 128)
    self.lstm = LSTM(512, return_sequences=True, return_state=True)

    # lstm의 모든 시퀀스 마다의 softmax를 적용시키기 위한 dense 레이어 선언
    self.dense = Dense(vocab_size, activation='softmax')

  def call(self, inputs):
    # decoder의 LSTM 호출은 데이터를 하나 하나씩 넣어주면서 timesteps에 의한 lstm 작동을 수동으로 해 준다.
    # inputs : 이전 스텝의 출력값, hidden_state, cell_state
    y, h, c = inputs # shifted_input, hidden_state, cell_state

    y = self.emb(y)
                                                 # 이전 스텝의 hidden state, cell state를 강제로 넣어준다.
    y, h, c = self.lstm(y, initial_state=[h, c]) # initial_state : LSTM의 hidden_state, cell_state를 강제로 설정
    
    return self.dense(y), h, c # softmax 결과, hidden state, cell state

## Seq2Seq 구현

In [ ]:
class Seq2Seq(tf.keras.Model):

  def __init__(self, sos="<sos>", eos="<eos>"):
    super(Seq2Seq, self).__init__()
    self.enc = Encoder()
    self.dec = Decoder()
    self.sos = sos
    self.eos = eos

  def call(self, inputs, training=False):
    '''
      inputs : feature, label
            : 훈련(training=True) 시에는 x, y를 동시에 받아와서 교사강요(Teacher Forcing)
            : 예측(training=False) 시에는 x만 받아와서 예측 수행
    '''
    if training:
      x, y = inputs # output, shifted input이 동시에 들어온다. ["이것은", "사과다", "<eos>"] , ["<sos>", "It's", "Apple", "<eos>"]

      # context vector
      h, c = self.enc(x)

      # y가 한꺼번에 들어간다. Teacher Forcing 수행
      y, _, _ = self.dec((y, h, c))

      return y
    else:
      x = inputs
      h, c = self.enc(x)

      # <sos> 텐서를 강제로 만들어 주기
      y = tf.convert_to_tensor(self.sos)
      y = tf.reshape(y, (1, 1)) # (1 배치, 1단어)

      # 예측값을 저장할 텐서를 미리 만들어 놓는다.
      total_seq = tf.TensorArray(tf.int32, 64) # 최대 64개의 공간을 미리 만들어 준다.

      for idx in tf.range(64):
        y, h, c = self.dec([y, h, c]) # 이 때 들어가는 y의 shape : (1, 1)
        y = tf.cast(tf.argmax(y, axis=-1), dtype=tf.int32)

        y = tf.reshape(y, (1, 1))
        total_seq = total_seq.write(idx, y)

        # y의 출력값이 eos라면 반복 중지
        if y == self.eos:
          break
      
      return tf.reshape(total_seq.stack(), (1, 64))

# 훈련, 테스트 루틴 정의

In [ ]:
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    output_labels = labels[:, 1:] # 제일 처음은 빼고 집어 넣는다. <sos>는 빼고~. 즉 <sos>가 없고 <eos>가 있고
    shifted_labels = labels[:, :-1] # <sos>를 포함하고 <eos>를 뺀다
    with tf.GradientTape() as tape:
        predictions = model([inputs, shifted_labels], training=True)
        loss = loss_object(output_labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(output_labels, predictions)

@tf.function
def test_step(model, inputs):
    return model(inputs, training=False)

In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from konlpy.tag import Okt
import random

In [ ]:
dataset_file = 'chatbot_data.csv'
okt = Okt()

with open(dataset_file, 'r') as file:
    lines = file.readlines()
    seq = [' '.join(okt.morphs(line)) for line in lines]

questions = seq[::2]
answers = ['\t ' + lines for lines in seq[1::2]]

num_sample = len(questions)

perm = list(range(num_sample))
random.seed(0)
random.shuffle(perm)

train_q = list()
train_a = list()
test_q = list()
test_a = list()

for idx, qna in enumerate(zip(questions, answers)):
    q, a = qna
    if perm[idx] > num_sample//5:
        train_q.append(q)
        train_a.append(a)
    else:
        test_q.append(q)
        test_a.append(a)

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=2000,
                                                  filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~')

tokenizer.fit_on_texts(train_q + train_a)

# 정수 인코딩
train_q_seq = tokenizer.texts_to_sequences(train_q)
train_a_seq = tokenizer.texts_to_sequences(train_a)

test_q_seq = tokenizer.texts_to_sequences(test_q)
test_a_seq = tokenizer.texts_to_sequences(test_a)

# 패딩
x_train = tf.keras.preprocessing.sequence.pad_sequences(train_q_seq,
                                                        value=0,
                                                        padding='pre',
                                                        maxlen=64)
y_train = tf.keras.preprocessing.sequence.pad_sequences(train_a_seq,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=65) # 앞에 sos를 붙여주고, 뒤에 eos가 붙어있는 상황. 훈련 시에 sos가 붙으면 eos가 빠지고, eos가 붙으면 sos가 빠지기 때문에 65개


x_test = tf.keras.preprocessing.sequence.pad_sequences(test_q_seq,
                                                       value=0,
                                                       padding='pre',
                                                       maxlen=64)
y_test = tf.keras.preprocessing.sequence.pad_sequences(test_a_seq,
                                                       value=0,
                                                       padding='post',
                                                       maxlen=65)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(1024)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(1).prefetch(1024)

# 모델 생성 및 훈련

In [ ]:
# 모델 생성
model = Seq2Seq(sos=tokenizer.word_index['\t'],
                eos=tokenizer.word_index['\n'])

# Loss, 최적화 정의
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# 훈련 평가방식 정의
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

## 학습 루프 작동 시키기

In [ ]:
for epoch in range(200):
    for seqs, labels in train_ds:
        train_step(model, seqs, labels, loss_object, optimizer, train_loss, train_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100))

    train_loss.reset_states()
    train_accuracy.reset_states()

Epoch 1, Loss: 3.1125128269195557, Accuracy: 83.22368621826172
Epoch 2, Loss: 0.604878306388855, Accuracy: 90.92261505126953
Epoch 3, Loss: 0.5685924291610718, Accuracy: 91.0753402709961
Epoch 4, Loss: 0.5580309629440308, Accuracy: 91.09100341796875
Epoch 5, Loss: 0.5391191840171814, Accuracy: 91.09492492675781
Epoch 6, Loss: 0.5338164567947388, Accuracy: 91.1614990234375
Epoch 7, Loss: 0.5396810173988342, Accuracy: 91.1732406616211
Epoch 8, Loss: 0.5245455503463745, Accuracy: 91.19673919677734
Epoch 9, Loss: 0.5149692893028259, Accuracy: 91.2202377319336
Epoch 10, Loss: 0.5025860071182251, Accuracy: 91.31422424316406
Epoch 11, Loss: 0.49026569724082947, Accuracy: 91.5413589477539
Epoch 12, Loss: 0.4730362892150879, Accuracy: 91.61576080322266
Epoch 13, Loss: 0.4583468735218048, Accuracy: 92.1483383178711
Epoch 14, Loss: 0.4438988268375397, Accuracy: 92.33631134033203
Epoch 15, Loss: 0.43541452288627625, Accuracy: 92.42637634277344
Epoch 16, Loss: 0.4296739399433136, Accuracy: 92.53211

## 테스트 루프 작동 시키기

In [ ]:
for test_seq, test_labels in test_ds:
    prediction = test_step(model, test_seq)
    test_text = tokenizer.sequences_to_texts(test_seq.numpy())
    gt_text = tokenizer.sequences_to_texts(test_labels.numpy())
    texts = tokenizer.sequences_to_texts(prediction.numpy())
    print('_')
    print('q: ', test_text[0].strip())
    print('a: ', gt_text[0].strip())
    print('p: ', texts[0].strip())

_
q:  여기 기프티콘 되죠
a:  네 현금영수증 해드릴까 요
p:  아메리카노 티라미슈 세트 가격 은 만 원 입니다
_
q:  네 에 테이크 아웃 도 가능한가요
a:  네 로 오시 면 테이크 아웃 잔 에 담아 드려요
p:  아뇨 현재 법적 으로 금지 하고 있어요
_
q:  아메리카노 톨 사이즈 로 주세요
a:  따뜻한 거 로 드릴 까요
p:  다른 건 필요 없으신 가요
_
q:  진동 을 따로 주시나요
a:  주 번호 로 드리겠습니다
p:  네 초코 머핀 이랑 치즈케이크 있습니다
_
q:  자리 있나요
a:  네 있습니다
p:  네 자리 있습니다
_
q:  그럼 루이보스 밀크 티 하나
a:  네 알겠습니다
p:  여기 진동 벨 가지 고 계시다가 울리면 주문 한 음료 가져가세요
_
q:  다음 에 무료 로 하고 엔 도장 찍어주세요
a:  네
p:  네 고객 님 결제 완료 되었습니다
_
q:  아메리카노 한 잔 에 얼마 죠
a:  입니다
p:  4000원 입니다
_
q:  얼마나
a:  바로 만들어 드릴게요
p:  오늘 딱 맞게 오셨네요
_
q:  카푸치노 는 로 주시 고 아메리카노 는 로
a:  네 더 없으세요
p:  결제 완료 되었습니다
_
q:  아메리카노 는 어떤 종류 가 있나요
a:  디카 페인 과 기본 아메리카노 2 종류 있습니다
p:  요즘 은 초코 케이크 가 제일 잘나가요
_
q:  카카오 페이 로 결제 가능한가요
a:  네 가능합니다
p:  네 가능합니다
_
q:  오늘 의 커피 는 커피 로 하나요 맛 이
a:  아 네 오늘 은 과테말라 커피 입니다
p:  오늘 딱 맞게 오셨네요
_
q:  머핀 은 뭐 가 제일
a:  블루베리 머핀 이 잘 나갑니다
p:  와이파이 비밀번호 는 종이 에 써져있습니다
_
q:  현금 영수증 해주세요
a:  네 번호 찍어주세요
p:  네 번호 찍어주세요
_
q:  둘 다 톨 사이즈 로 주세요
a:  여기 서 드시고 요
p:  주문 이 따뜻하게 드릴 까요
_
q:  아이스 아메리카노 한 잔 가능한가요
a:  네 가능합